In [ ]:
import yfinance as yf
import pandas as pd
import requests
import matplotlib.pyplot as plt
from datetime import datetime
import time
import csv

# Telegram config (disabled for notebook testing)
BOT_TOKEN = "8120831257:AAHR4QZyDRyH6Am0bK6OhDiJUPyqc-PCitU"
CHAT_ID = "940705270"


In [2]:
def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {
        "chat_id": CHAT_ID,
        "text": message
    }
    try:
        response = requests.post(url, data=payload)
        return response.status_code == 200
    except:
        return False


In [3]:
def log_signal(ticker, signal, price):
    with open("trade_signals_log.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now(), ticker, signal, price])
    print(f"📝 Logged: {ticker} | {signal} | ₹{price:.2f}")


In [4]:
def fetch_intraday_data(ticker):
    print(f"🔍 Fetching data for {ticker}...")
    df = yf.download(tickers=ticker, period="1d", interval="1m", progress=False)
    df.dropna(inplace=True)
    print(f"✅ {ticker} data shape: {df.shape}")
    display(df.tail(5))
    return df


In [5]:
def add_indicators(df):
    df = df.copy()

    # EMA
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()

    # RSI
    delta = df["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26
    df["MACD_signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

    df.dropna(inplace=True)
    print("📊 Basic indicators added.")
    display(df.tail(5))
    return df


In [6]:
def add_advanced_indicators(df):
    df = df.copy()

    df["VWAP"] = (df["Volume"] * (df["High"] + df["Low"] + df["Close"]) / 3).cumsum() / df["Volume"].cumsum()

    # Bollinger Bands
    df["BB_MID"] = df["Close"].rolling(window=20).mean()
    df["BB_STD"] = df["Close"].rolling(window=20).std()
    df["BB_UPPER"] = df["BB_MID"] + 2 * df["BB_STD"]
    df["BB_LOWER"] = df["BB_MID"] - 2 * df["BB_STD"]

    # ATR
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Close"].shift())
    df["L-PC"] = abs(df["Low"] - df["Close"].shift())
    df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis=1)
    df["ATR"] = df["TR"].rolling(window=14).mean()
    df.drop(["H-L", "H-PC", "L-PC", "TR"], axis=1, inplace=True)

    df.dropna(inplace=True)
    print("🔬 Advanced indicators added.")
    display(df.tail(5))
    return df


In [7]:
def generate_advanced_signal(df):
    if len(df) < 2:
        return "HOLD", None

    last = df.iloc[[-1]]  # Keep as DataFrame to support .iloc[0]
    prev = df.iloc[[-2]]

    try:
        rsi = float(last["RSI"].iloc[0])
        macd = float(last["MACD"].iloc[0])
        macd_sig = float(last["MACD_signal"].iloc[0])
        price = float(last["Close"].iloc[0])
        vwap = float(last["VWAP"].iloc[0])

        # Relaxed condition for testing
        if rsi < 45 and macd > macd_sig and price < vwap:
            return "BUY", price
        elif rsi > 60 and macd < macd_sig and price > vwap:
            return "SELL", price
        return "HOLD", price

    except Exception as e:
        print("⚠️ Signal error:", e)
        return "HOLD", None





In [9]:
# List of volatile tickers
tickers = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS",
    "SBIN.NS", "AXISBANK.NS", "KOTAKBANK.NS", "ITC.NS", "HINDUNILVR.NS",
    "LT.NS", "BAJFINANCE.NS", "ADANIENT.NS", "ADANIGREEN.NS", "WIPRO.NS",
    "POWERGRID.NS", "ONGC.NS", "ULTRACEMCO.NS", "ASIANPAINT.NS", "MARUTI.NS"
]


all_signals = []  # For Telegram summary

for ticker in tickers:
    try:
        print(f"\n🔍 Fetching: {ticker}")
        df = fetch_intraday_data(ticker)
        df = add_indicators(df)
        df = add_advanced_indicators(df)

        df.to_csv(f"{ticker}_indicators.csv")
        print(f"💾 Saved {ticker}_indicators.csv")

        signal, price = generate_advanced_signal(df)

        if price:
            signal_msg = f"{signal} signal for {ticker} at ₹{price:.2f}"
        else:
            signal_msg = f"{signal} signal for {ticker} (price N/A)"

        print(signal_msg)
        all_signals.append(signal_msg)

        log_signal(ticker, signal, price)

    except Exception as e:
        error_msg = f"❌ Error for {ticker}: {e}"
        print(error_msg)
        all_signals.append(error_msg)

# Send all results in one combined message
final_msg = "📡 Signal Summary:\n\n" + "\n".join(all_signals)
send_telegram_message(final_msg)





🔍 Fetching: RELIANCE.NS
🔍 Fetching data for RELIANCE.NS...


C:\Users\Sangharsh\AppData\Local\Temp\ipykernel_21720\2642469662.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers=ticker, period="1d", interval="1m", progress=False)


✅ RELIANCE.NS data shape: (375, 5)


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Datetime,,,,,
2025-07-30 09:55:00+00:00,1409.800049,1410.500000,1409.199951,1409.400024,48569
2025-07-30 09:56:00+00:00,1410.099976,1410.900024,1409.199951,1409.800049,43881
2025-07-30 09:57:00+00:00,1410.500000,1411.000000,1410.199951,1410.199951,40870
2025-07-30 09:58:00+00:00,1411.099976,1412.099976,1410.099976,1411.000000,33653
2025-07-30 09:59:00+00:00,1411.000000,1411.199951,1409.199951,1410.500000,8472


📊 Basic indicators added.


...

🔬 Advanced indicators added.


Price,Close,High,Low,Open,Volume,EMA_20,RSI,MACD,MACD_signal,VWAP,BB_MID,BB_STD,BB_UPPER,BB_LOWER,ATR
Ticker,MARUTI.NS,MARUTI.NS,MARUTI.NS,MARUTI.NS,MARUTI.NS,,,,,,,,,,
Datetime,,,,,,,,,,,,,,,
2025-07-30 09:55:00+00:00,12619.0,12624.0,12618.0,12624.0,876,12617.191921,47.058824,2.877244,3.394215,12581.521445,12618.35,2.739093,12623.828186,12612.871814,6.142857
2025-07-30 09:56:00+00:00,12610.0,12621.0,12610.0,12620.0,657,12616.506976,35.714286,1.972468,3.109865,12581.632000,12617.90,3.307090,12624.514179,12611.285821,6.428571
2025-07-30 09:57:00+00:00,12609.0,12615.0,12606.0,12615.0,559,12615.792026,35.714286,1.161348,2.720162,12581.714770,12617.60,3.816860,12625.233720,12609.966280,6.642857
2025-07-30 09:58:00+00:00,12595.0,12616.0,12595.0,12616.0,651,12613.811833,28.846154,-0.604189,2.055292,12581.783463,12616.35,6.285112,12628.920224,12603.779776,7.714286
2025-07-30 09:59:00+00:00,12612.0,12612.0,12586.0,12598.0,254,12613.639277,49.230769,-0.624434,1.519346,12581.811899,12616.05,6.345118,12628.740237,12603.359763,9.214286


💾 Saved MARUTI.NS_indicators.csv
HOLD signal for MARUTI.NS at ₹12612.00
📝 Logged: MARUTI.NS | HOLD | ₹12612.00


C:\Users\Sangharsh\AppData\Local\Temp\ipykernel_21720\3227616758.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(last["Close"].iloc[0])


True